In [ ]:
!mkdir /content/Bing_data

In [ ]:
#/content/drive/MyDrive/Sync/BingImage-ACACIA_RIDGE_QLD7.zip

import zipfile

zip_file = "/content/drive/MyDrive/Bing_data/Bing_data06.zip"

try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall("/content/Bing_data/")
        print("Extracted all")
except:
    print("Invalid file")

In [ ]:
!mkdir /content/Google_data

In [ ]:
#/content/drive/MyDrive/Sync/BingImage-ACACIA_RIDGE_QLD7.zip

import zipfile

zip_file = "/content/drive/MyDrive/Google_data/Google_data06.zip"

try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall("/content/Google_data/")
        print("Extracted all")
except:
    print("Invalid file")

In [ ]:
cd /content/drive/MyDrive/image_tabular-master

In [ ]:
import fastai; fastai.__version__

'1.0.61'

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.tabular import *
from image_tabular.core import *
from image_tabular.dataset import *
from image_tabular.model import *
from image_tabular.metric import *

In [ ]:
bs = 32
imsize = 256

np.random.seed(2)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [ ]:
data_path = Path("/content/sample_data/")

In [ ]:
house_price_df = pd.read_csv('/content/drive/MyDrive/QUTREProject/Brisbane_suburbs_02_sales_2018_property_GB_Prop_dist16_new2.csv', index_col=0)
house_price_df

In [ ]:
house_price_df.columns

In [ ]:
test_df = house_price_df.loc[(house_price_df['suburb_id_x']=='QLD1346') | (house_price_df['suburb_id_x']=='QLD627') | (house_price_df['suburb_id_x']=='QLD24') | \
                         (house_price_df['suburb_id_x']=='QLD3231') | (house_price_df['suburb_id_x']=='QLD32') | (house_price_df['suburb_id_x']=='QLD510') | \
                         (house_price_df['suburb_id_x']=='QLD545') | (house_price_df['suburb_id_x']=='QLD718') | (house_price_df['suburb_id_x']=='QLD389') ]
test_df

In [ ]:
# train_df=house_price_df.sample(frac=0.8,random_state=200) #random state is a seed value
# test_df=house_price_df.drop(train_df.index)
train_df=house_price_df.drop(test_df.index)
train_df.reset_index()
test_df.reset_index()

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
test_df.reset_index(drop=True, inplace=True)
test_df

In [ ]:
# Check the length of the dataset
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
print(train_df.columns)

In [ ]:
train_df.describe()

In [ ]:
# # Split our variables into target, categorical and continuous variables
# dep_var = 'price_value'

# # cat_names = ['suburb_id_x']
# cat_names = ['street_id_x']
# cont_names = ['propertyFeatures_bedrooms_x', 'propertyFeatures_bathrooms_x', 'propertyFeatures_carparks_x', 'landDetails_propertyArea_x', 'school_dist'\
#               , 'hospital_dist', 'uni_dist', 'park_dist', 'mall_dist', 'CBD_dist', 'First_year_transaction']

# print(cat_names)
# print(cont_names)

In [ ]:
tfms = get_transforms(do_flip=False, max_rotate=5, max_zoom=1.05)

procs = [Normalize]

size = 256

In [ ]:
# idx for validation, shared by image and tabular data
val_idx = get_valid_index(train_df)
len(val_idx)

In [ ]:
# load image data using train_df and prepare fastai LabelLists
image_path = Path("/content/Google_data/")

image_data = (ImageList.from_df(train_df, path=image_path, cols="image_Google")
              .split_by_idx(val_idx)
              .label_from_df(cols="price_value", label_cls=FloatList, log=True)
              .transform(tfms, size=size))

# add test data so that we can make predictions
test_image_data = ImageList.from_df(test_df, path=image_path, cols="image_Google")
image_data.add_test(test_image_data)

In [ ]:
# show one example image
print(image_data.train[0][1])
image_data.train[0][0]

In [ ]:
dep_var = 'price_value'
# cat_names = ['suburb_id_x']
cat_names = []
# cat_names = ['street_id_x']
cont_names = ['propertyFeatures_bedrooms_x', 'propertyFeatures_bathrooms_x', 'propertyFeatures_carparks_x', 'landDetails_propertyArea_x', 'school_dist'\
              , 'hospital_dist', 'uni_dist', 'park_dist', 'mall_dist', 'CBD_dist', 'First_year_transaction', 'highway_dist', 'bus_stop_dist', \
       'ferry_terminal_dist', 'railway_station_dist', 'number_transaction', \
       'crime_rate_2017', 'state_school_percentage', \
       'class_size_achievement_pct_2017', 'Density', 'Score2017']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
tab_data = (TabularList.from_df(train_df, path=data_path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_idx)
                           .label_from_df(cols=dep_var, label_cls=FloatList, log=True))

# add test
tab_data.add_test(TabularList.from_df(test_df, cat_names=cat_names, cont_names=cont_names,
                                      processor = tab_data.train.x.processor))

In [ ]:
tab_data.train[0]

In [ ]:
integrate_train, integrate_valid, integrate_test = get_imagetabdatasets(image_data, tab_data)

In [ ]:
# package train, valid, and test datasets into a fastai databunch
bs = 64

db = DataBunch.create(integrate_train, integrate_valid, integrate_test, path=data_path, bs=bs)
db

In [ ]:
# image normalization with imagenet_stats
db.norm, db.denorm = normalize_funcs_image_tab(*imagenet_stats)
db.add_tfm(db.norm)

In [ ]:
# check the shape of one batch
x, y = next(iter(db.train_dl))
len(x)

In [ ]:
# images
x[0].shape

In [ ]:
# categorical and continuous tabular data
x[1][0].shape, x[1][1].shape

In [ ]:
# targets
y.shape

In [ ]:
# cnn model for images, use Resnet50 as an example
#cnn_arch = models.resnet50
cnn_arch = models.resnet34

# cnn_out_sz is the output size of the cnn model that will be concatenated with tabular model output
cnn_out_sz = 256

# use fastai functions to get a cnn model
image_data_db = image_data.databunch()
image_data_db.c = cnn_out_sz
cnn_learn = cnn_learner(image_data_db, cnn_arch, ps=0.2)
cnn_model = cnn_learn.model

In [ ]:
# get embedding sizes of categorical data
emb_szs = tab_data.train.get_emb_szs()

# output size of the tabular model that will be concatenated with cnn model output
tab_out_sz = 1

# use fastai functions to get a tabular model
tabular_model = TabularModel(emb_szs, len(cont_names), out_sz=tab_out_sz, layers=[], ps=0.2)
tabular_model

In [ ]:
# use gpu by default if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [ ]:
torch.cuda.is_available()

In [ ]:
# get an integrated model that combines the two components and concatenate their outputs
# which will pass through additional fully connected layers
integrate_model = CNNTabularModel(cnn_model,
                                  tabular_model,
                                  layers = [cnn_out_sz + tab_out_sz, 1],
                                  ps=0.2,
                                  out_sz=1).to(device)

In [ ]:
# check model output dimension, should be (bs, 2)
integrate_model(*x).shape

In [ ]:
# adjust loss function weight because the dataset is extremely unbalanced
weights = [1/(1-train_df["price_value"].mean()), 1/train_df["price_value"].mean()]
#loss_func = CrossEntropyFlat(weight=torch.FloatTensor(weights).to(device))
#loss_func = CrossEntropyFlat(weight=torch.FloatTensor(weights).to(device))
class L1LossFlat(nn.SmoothL1Loss):
    def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
        return super().forward(input.view(-1), target.view(-1))

In [ ]:
# package everything in a fastai learner, add auc roc score as a metric
#learn = Learner(db, integrate_model, metrics=[accuracy, ROCAUC()], loss_func=loss_func)
learn = Learner(db, integrate_model, metrics=[root_mean_squared_error, r2_score])
learn.loss_func = MSELossFlat()

In [ ]:
# organize layer groups in order to use differential learning rates provided by fastai
# the first two layer groups are earlier layers of resnet
# the last layer group consists of the fully connected layers of cnn model, tabular model,
# and final fully connected layers for the concatenated data
learn.layer_groups = [nn.Sequential(*flatten_model(cnn_learn.layer_groups[0])),
                      nn.Sequential(*flatten_model(cnn_learn.layer_groups[1])),
                      nn.Sequential(*(flatten_model(cnn_learn.layer_groups[2]) +
                                      flatten_model(integrate_model.tabular_model) +
                                      flatten_model(integrate_model.layers)))]

In [ ]:
# find learning rate to train the last layer group first
learn.freeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
# train
min_grad_lr = 2e-1 # learn.recorder.min_grad_lr
learn.fit_one_cycle(13, min_grad_lr)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('TabularCNN_resnet34_freeze')
learn.load('TabularCNN_resnet34_freeze')

In [ ]:
# make predictions for the test set
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
print(preds)

In [ ]:
print(y)

In [ ]:
# submit predictions to kaggle
labels = [np.exp(p[0].data.item()) for p in preds]
submission = pd.DataFrame({'Id': test_df['id_combined'], 'Price': test_df['price_value']})
submission["Predicted"] = labels
submission.to_csv(data_path/'image_tab_freeze.csv', index=False)
submission.head()

In [ ]:
import pandas
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels.formula.api as sm
import scipy.stats as stats

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['Price'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Price'].corr(submission['Predicted'])))

In [ ]:
import numpy as np
Y_true = submission['Price']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
dfCopyf = test_df.copy()
dfCopyf["Predicted"] = labels

In [ ]:
# dfCopy.reset_index(drop=True, inplace=True)
dfCopyf.to_csv('/content/drive/MyDrive/QUTREProject/2018_test_results_Hedonic_CNN_Fusion_Google256_Exp2_new2_freeze.csv',index=False)
dfCopyf.to_pickle('/content/drive/MyDrive/QUTREProject/2018_test_results_Hedonic_CNN_Fusion_Google256_Exp2_new2_freeze.pkl')

In [ ]:
# unfreeze all layer groups to train the entire model using differential learning rates
learn.unfreeze()
learn.fit_one_cycle(5, slice(min_grad_lr, 1e-5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('TabularCNN_resnet34_unfreeze')
learn.load('TabularCNN_resnet34_unfreeze')

In [ ]:
# make predictions for the test set
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
print(preds)

In [ ]:
print(y)

In [ ]:
# submit predictions to kaggle
labels = [np.exp(p[0].data.item()) for p in preds]
submission = pd.DataFrame({'Id': test_df['id_combined'], 'Price': test_df['price_value'], 'street_id': test_df['street_id_x']})
submission["Predicted"] = labels
submission.to_csv(data_path/'image_tab_unfreeze.csv', index=False)
submission.head()

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['Price'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Price'].corr(submission['Predicted'])))

In [ ]:
import numpy as np
Y_true = submission['Price']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
# # embeddings = cnn_learn.predict(DatasetType.Test)
# embeddings, y = cnn_learn.get_preds(DatasetType.Test)

In [ ]:
# print(embeddings)

In [ ]:
# print(embeddings.shape)

In [ ]:
# print(embeddings[5])

In [ ]:
# embeddings01 = embeddings[0:3]

In [ ]:
# print(embeddings01)

In [ ]:
# test_df

In [ ]:
dfCopy = test_df.copy()
dfCopy

In [ ]:
dfCopy["Predicted"] = labels
dfCopy

In [ ]:
# dfCopy.reset_index(drop=True, inplace=True)
dfCopy.to_csv('/content/drive/MyDrive/QUTREProject/2018_test_results_Hedonic_CNN_Fusion_Google256_Exp2_new2_unfreeze.csv',index=False)
dfCopy.to_pickle('/content/drive/MyDrive/QUTREProject/2018_test_results_Hedonic_CNN_Fusion_Google256_Exp2_new2_unfreeze.pkl')

In [ ]:
# dfCopy['Index_x'] = 0
# dfCopy

In [ ]:
# class_price_01 = 1000000
# class_price_02 = 800000
# class_price_03 = 600000
# class_price_04 = 400000
# class_price_05 = 200000

# for ii in range(len(dfCopy)):
#   if dfCopy['price_value'][ii] > class_price_01:
#         dfCopy.iloc[[ii],[40]] = 6
#   elif (dfCopy['price_value'][ii] <= class_price_01) & (dfCopy['price_value'][ii] > class_price_02):
#         dfCopy.iloc[[ii],[40]] = 5
#   elif (dfCopy['price_value'][ii] <= class_price_02) & (dfCopy['price_value'][ii] > class_price_03):
#         dfCopy.iloc[[ii],[40]] = 4
#   elif (dfCopy['price_value'][ii] <= class_price_03) & (dfCopy['price_value'][ii] > class_price_04):
#         dfCopy.iloc[[ii],[40]] = 3
#   elif (dfCopy['price_value'][ii] <= class_price_04) & (dfCopy['price_value'][ii] > class_price_05):
#         dfCopy.iloc[[ii],[40]] = 2
#   else:
#         pass

#   # dfCopy['Index_x'][ii] = ii
# dfCopy

In [ ]:
# class_price_01 = 1000000
# class_price_02 = 800000
# class_price_03 = 600000
# class_price_04 = 400000
# class_price_05 = 200000

# for ii in range(len(dfCopy)):
#   if submission['Predicted'][ii] > class_price_01:
#         dfCopy.iloc[[ii],[41]] = 6
#   elif (submission['Predicted'][ii] <= class_price_01) & (submission['Predicted'][ii] > class_price_02):
#         dfCopy.iloc[[ii],[41]] = 5
#   elif (submission['Predicted'][ii] <= class_price_02) & (submission['Predicted'][ii] > class_price_03):
#         dfCopy.iloc[[ii],[41]] = 4
#   elif (submission['Predicted'][ii] <= class_price_03) & (submission['Predicted'][ii] > class_price_04):
#         dfCopy.iloc[[ii],[41]] = 3
#   elif (submission['Predicted'][ii] <= class_price_04) & (submission['Predicted'][ii] > class_price_05):
#         dfCopy.iloc[[ii],[41]] = 2
#   else:
#         pass

#   # dfCopy['Index_x'][ii] = ii
# dfCopy

In [ ]:
# dfCopy.set_index('Index_x', inplace=True)
# dfCopy

In [ ]:
# # QLD125992
# dfCopy["Index_x"] = 0
# dfCopy.loc[dfCopy["suburb_id_x"] == 'QLD545', "Index_x"] = 1
# dfCopy.loc[dfCopy["suburb_id_x"] == 'QLD2303', "Index_x"] = 2
# dfCopy.loc[dfCopy["suburb_id_x"] == 'QLD2330', "Index_x"] = 3
# dfCopy.loc[dfCopy["suburb_id_x"] == 'QLD1082', "Index_x"] = 4
# dfCopy.loc[dfCopy["suburb_id_x"] == 'QLD1834', "Index_x"] = 5


In [ ]:
# # street_id_test = dfCopy['suburb_id_x']
# street_id_test = dfCopy['Index_x']
# street_id_test

In [ ]:
# print(street_id_test[0][3:])

In [ ]:
# for ii in range(len(street_id_test)):
#   street_id_test[ii] = int(street_id_test[ii][3:])
# street_id_test

In [ ]:
# embeddings01 = embeddings[0:500]

In [ ]:
# from sklearn.manifold import TSNE

# tsne_embeddings = TSNE(random_state=2, verbose=1,
#                     metric='euclidean').fit_transform(embeddings)

In [ ]:
# print(tsne_embeddings)

In [ ]:
# dataframe=pd.DataFrame(tsne_embeddings, columns=['a','b'])
# dataframe

In [ ]:
# dataframe.to_csv('/content/drive/MyDrive/QUTREProject/tsne_embeddings_2018_T-1_Google256_CNN_HE_Fusion_Colab.csv')

In [ ]:
# fig = plt.figure(figsize=[12, 12])
# ax = fig.add_subplot(1, 1, 1)
# # ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1],c = street_id_test);
# sc = ax.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c=street_id_test, cmap='tab10')
# ax.legend(*sc.legend_elements(), title='clusters')

In [ ]:
# from matplotlib.offsetbox import OffsetImage, AnnotationBbox
# from matplotlib.cbook import get_sample_data
# import PIL
# from PIL import Image


# def plot_images_in_2d(x, y, image_paths, axis=None, zoom=1):
#     if axis is None:
#         axis = plt.gca()
#     x, y = np.atleast_1d(x, y)
#     for x0, y0, image_path in zip(x, y, image_paths):
#         image = Image.open(image_path)
#         image.thumbnail((100, 100), Image.ANTIALIAS)
#         img = OffsetImage(image, zoom=zoom)
#         anno_box = AnnotationBbox(img, (x0, y0),
#                                   xycoords='data',
#                                   frameon=False)
#         axis.add_artist(anno_box)
#     axis.update_datalim(np.column_stack([x, y]))
#     axis.autoscale()

In [ ]:
# def show_tsne(x, y, selected_filenames):
#     fig, axis = plt.subplots()
#     fig.set_size_inches(22, 22, forward=True)
#     plot_images_in_2d(x, y, selected_filenames, zoom=0.3, axis=axis)
#     plt.show()

In [ ]:
# selected_filenames = '/content/Bing_data/' + dfCopy['image_Bing']
# selected_filenames

In [ ]:
# selected_filenames = '/content/Google_data/' + dfCopy['image_Google']
# selected_filenames

In [ ]:
# show_tsne(tsne_embeddings[:, 0], tsne_embeddings[:, 1], selected_filenames)

In [ ]:
# def tsne_to_grid_plotter_manual(x, y, selected_filenames):
#     S = 2000
#     s = 100
#     x = (x - min(x)) / (max(x) - min(x))
#     y = (y - min(y)) / (max(y) - min(y))
#     x_values = []
#     y_values = []
#     filename_plot = []
#     x_y_dict = {}
#     for i, image_path in enumerate(selected_filenames):
#         a = np.ceil(x[i] * (S - s))
#         b = np.ceil(y[i] * (S - s))
#         a = int(a - np.mod(a, s))
#         b = int(b - np.mod(b, s))
#         if str(a) + "|" + str(b) in x_y_dict:
#             continue
#         x_y_dict[str(a) + "|" + str(b)] = 1
#         x_values.append(a)
#         y_values.append(b)
#         filename_plot.append(image_path)
#     fig, axis = plt.subplots()
#     fig.set_size_inches(22, 22, forward=True)
#     plot_images_in_2d(x_values, y_values, filename_plot, zoom=.58, axis=axis)
#     plt.show()

In [ ]:
# tsne_to_grid_plotter_manual(tsne_embeddings[:, 0], tsne_embeddings[:, 1],
#                             selected_filenames)